In [1]:
import numpy as np

def RELU(x):
    return np.maximum(0,x)

def relu_der(z):
    return 1.*(z>0)

def softmax(x):
    return np.exp(x)/np.sum(np.exp(x))

def softmax_der(x):
    prob = softmax(x)
#     print(np.shape(prob))
    r = np.matmul(prob.T, (np.ones_like(prob)-prob))
#     print(np.shape(r))
    return r

def oneHotEncode(y,softmaxClasses):
        yactual = np.matrix(np.zeros(softmaxClasses)).T
        yactual[y] = 1
        return yactual    
    
def crossEntropy(ypred,y,softmaxClasses):
    #one hot encode
    yactual = oneHotEncode(y,softmaxClasses)
    l = np.log(ypred)
    return -np.sum(l.T*yactual)


def delta_cross_entropy(y, X):
          m = y.shape[0]
          grad = softmax(X)
          idx = np.where(grad == y.T*grad)
          grad[idx] = grad[idx] - 1
          grad = grad/m
          return grad



class neuralNetwork():    
    FCreluLayers = 0  
    Weights = []
    Biases = []
    softmax = False
    softmaxClasses = 0
    loss = []
    delta = [[]];    dW = [[]];    db = [[]]
    act = []
    logits = [[]]
    def FullyConnectedreluLayers(self, values):
        assert len(values) == self.FCreluLayers+1, "Number of fully connected layers are not the same."
        self.Weights.append([])
        self.Biases.append([])
        for i in range(1, self.FCreluLayers+1):            
            self.Weights.append(np.random.normal(0, 0.1, ([values[i-1], values[i]])))
            self.Biases.append(np.random.normal(0, 0.1, ([values[i], 1])))
            self.delta.append([]);    self.dW.append([]);    self.db.append([])
               
    
    def softmaxLayer(self):
        previous = self.Weights[-1].shape[1]
        self.Weights.append(np.random.normal(0,0.1,[previous, self.softmaxClasses]))
        self.Biases.append(np.random.normal(0, 0.1, (self.softmaxClasses, 1)))
        self.delta.append([]);    self.dW.append([]);    self.db.append([])
        
    def calcLoss(self, ypred, yactual):
        if self.softmax == True:
            return crossEntropy(ypred,yactual, self.softmaxClasses)
    
    def feedForward(self,X):
        self.act.append(X.T)
        for i in range(1, self.FCreluLayers+1):
#             print("weights shape: ", np.shape(self.Weights[i].T), " act: ", np.shape(self.act[i-1]))
            self.logits.append(np.matmul(self.Weights[i].T, self.act[i-1]) + self.Biases[i])
            self.act.append( RELU(self.logits[-1]) )

        if self.softmax == True:
            self.logits.append(np.matmul(self.Weights[-1].T, self.act[-1]) + self.Biases[-1])
            self.act.append(softmax(self.logits[-1]))

#         print("Weights: ", [np.shape(self.Weights[i]) for i in range(len(self.Weights))])
#         print("act shapes: ", [np.shape(self.act[i]) for i in range(len(self.act))])
#         print("logits shapes: ", [np.shape(self.logits[i]) for i in range(len(self.logits))])
        return self.act[-1]
    
    def backprop(self, ypred, yact):
        L = len(self.delta)-1
        yact = oneHotEncode(yact, self.softmaxClasses)
#         print(yact)
        self.delta[L] = delta_cross_entropy(ypred,yact)
#         print("delta_cross_entropy: ", np.sum(self.delta[L]))
#         self.delta[L] = np.subtract(yact, ypred)
#         print(np.shape(self.delta[L]))
        self.db[L] = self.delta[L].copy()
        self.dW[L] = self.act[L-1] * self.delta[L].T

        WxD = self.Weights[L] * self.delta[L]
        self.delta[L-1] = np.multiply(softmax_der(self.logits[L-1]), WxD.T)
        
        L = 2

        self.db[L] = self.delta[L].copy()
        self.dW[L] = self.act[L-1] * self.delta[L]
        bsd = self.Weights[L]* self.delta[L].T
#         print("weights: ", np.shape(self.Weights[L]), " delta: ", np.shape(self.delta[L]))
        reluOut = relu_der(self.logits[L-1])
#         print("bsd: ", np.shape(bsd))
#         print(" logit to reul shape: ", np.shape(self.logits[L-1]))
        self.delta[L-1] = np.multiply(reluOut, bsd)
#         print("new delta: ", np.shape(self.delta[L-1]))
        
        
        L = 1        

        self.db[L] = self.delta[L].copy()
        self.dW[L] = self.act[L-1] * self.delta[L].T
            
        alpha = 0.01
        for l in range(1,len(self.Weights)):            
            self.Weights[l] = self.Weights[l] - alpha * (-1*self.dW[l])
            self.Biases[l] = self.Biases[l] - alpha * (-1*self.db[l])
# mnist.loss.append(mnist.calcLoss(ypred, yt))
# print(mnist.loss[-1])

# print([np.shape(mnist.Weights[i]) for i in range(len(mnist.Weights))])  
# print(mnist.feedForward(np.matrix([0,1,2,3,4,5]), 3 ))
# mnist.layerWeights[-1].shape[1]


In [2]:
from sklearn.datasets import load_digits
digits = load_digits()
print("listing digits: ", list(digits))
print("Number of samples: ", digits.data.shape[0])
print("Size of each sample: ", digits.data.shape[1])
print("Listing target names: ", list(digits.target_names))

y = np.matrix(digits.target).T
X = np.matrix(digits.data)
M = X.shape[0]
N = X.shape[1]
print("number of sample: ",M, " each sample: ",N)

listing digits:  ['data', 'target', 'target_names', 'images', 'DESCR']
Number of samples:  1797
Size of each sample:  64
Listing target names:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
number of sample:  1797  each sample:  64


In [3]:
def normalize(X):    
    return X/255

X = normalize(X)

In [4]:
print("Image at random index 3: ")
import matplotlib.pyplot as plt 
plt.matshow(digits.images[3]) 
plt.show()

Image at random index 3: 


<Figure size 480x480 with 1 Axes>

In [ ]:
mnist = neuralNetwork()
mnist.__class__.FCreluLayers = 2 #excluding the softmax
mnist.__class__.softmax = True
mnist.__class__.softmaxClasses = 10
N = 64
h1 = 24
h2 = 16
mnist.FullyConnectedreluLayers([N,h1,h2])
mnist.softmaxLayer()

# M = 10
max_iter = 1000
for iter in range(0, max_iter):
    loss_this_iter = 0
    order = np.random.permutation(M)
    for i in range(0,M):
        
        x_this = X[order[i],:]
#         print(type(x_this))
        y_this = y[order[i],0]
#         print(y_this)
#         print(x_this.shape)
#         print(np.ones(64).shape)
#         np.squeeze(np.asarray(x_this)).T 
        ypred = mnist.feedForward(x_this)
        # print(np.shape(ypred), np.zeros(10))
        
        loss_this_iter += mnist.calcLoss(ypred, y_this)
        mnist.backprop(ypred,y_this)
    print('Iteration %d loss %f' % (iter, loss_this_iter))

/home/prithvi/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:34: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


Iteration 0 loss 145944.650694
Iteration 1 loss 146049.000659
Iteration 2 loss 146122.038266
Iteration 3 loss 146331.173656
Iteration 4 loss 146805.606024
Iteration 5 loss 147913.330356
Iteration 6 loss 150327.666403
Iteration 7 loss 155553.783121
Iteration 8 loss 166160.803033
Iteration 9 loss 185119.181665
Iteration 10 loss 214047.660758
Iteration 11 loss 251714.505749
Iteration 12 loss 297178.750076
Iteration 13 loss 349650.048678
Iteration 14 loss 409082.953135
Iteration 15 loss 475762.028403
Iteration 16 loss 550152.833741
Iteration 17 loss 632777.263419
Iteration 18 loss 723324.186886
Iteration 19 loss 822832.496913
Iteration 20 loss 931637.444480
Iteration 21 loss 1049660.068416
Iteration 22 loss 1177564.604203
Iteration 23 loss 1315735.723830
Iteration 24 loss 1464430.008528
Iteration 25 loss 1624388.307059
Iteration 26 loss 1795163.296225
Iteration 27 loss 1978292.327578
Iteration 28 loss 2172632.716823
Iteration 29 loss 2379598.930777
Iteration 30 loss 2599969.274304
Iteratio

/home/prithvi/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
/home/prithvi/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log
/home/prithvi/anaconda3/envs/pytorch/lib/python3.7/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


Iteration 60 loss nan
Iteration 61 loss nan
Iteration 62 loss nan
Iteration 63 loss nan
Iteration 64 loss nan
Iteration 65 loss nan
Iteration 66 loss nan
Iteration 67 loss nan
Iteration 68 loss nan
Iteration 69 loss nan
Iteration 70 loss nan
Iteration 71 loss nan
Iteration 72 loss nan
Iteration 73 loss nan
Iteration 74 loss nan
Iteration 75 loss nan
Iteration 76 loss nan
Iteration 77 loss nan
Iteration 78 loss nan
Iteration 79 loss nan
Iteration 80 loss nan
Iteration 81 loss nan
Iteration 82 loss nan
Iteration 83 loss nan
Iteration 84 loss nan
Iteration 85 loss nan
Iteration 86 loss nan
Iteration 87 loss nan
Iteration 88 loss nan
Iteration 89 loss nan
Iteration 90 loss nan
Iteration 91 loss nan
Iteration 92 loss nan
Iteration 93 loss nan
Iteration 94 loss nan
Iteration 95 loss nan
Iteration 96 loss nan
Iteration 97 loss nan
Iteration 98 loss nan
Iteration 99 loss nan
Iteration 100 loss nan
Iteration 101 loss nan
Iteration 102 loss nan
Iteration 103 loss nan
Iteration 104 loss nan
Itera

In [ ]:

# r = np.array([1,0,0])
# logits = np.array([0.6,0,0.5])
# logits_for_answers = logits[np.arange(len(logits)),r]

In [ ]:
np.shape(np.matrix(np.zeros(5)).T)

In [ ]:
def oneHotEncode(y,softmaxClasses):
        yactual = np.matrix(np.zeros(softmaxClasses)).T
        yactual[y] = 1
        return yactual
    
def crossEntropy(ypred,y,softmaxClasses):
    """Compute crossentropy from logits[batch,n_classes] and ids of correct answers"""
    #one hot encode
    yactual = oneHotEncode(y,softmaxClasses)
    print(yactual)
    l = np.log(ypred)
    print(l)
    return - np.sum(l*yactual)/yactual.shape[0]

crossEntropy(np.array([1.6,3.4,1.4,4]), 2, 4)

In [ ]:
def delta_cross_entropy(y, X):
          m = y.shape[0]
          grad = softmax(X)
          idx = np.where(grad == y*grad)
          grad[idx] = grad[idx] - 1
          grad = grad/m
          return grad
delta_cross_entropy(np.array([4,3,2,1]), np.array([3,4,12,4]))

In [ ]:
softmax_der(np.ones(4))